This script retrieves all pickup and dropoff locations from train.csv, and creates a web spider which requests these direction information from OSRM. The script outputs the result to direction.csv.

In [7]:
import urllib.request
import requests
import json
import pandas as pd


taxi=pd.read_csv(r"train.csv")


taxi['pickup_location']=taxi['pickup_longitude'].astype(str)+','+taxi['pickup_latitude'].astype(str)
taxi['dropoff_location']=taxi['dropoff_longitude'].astype(str)+','+taxi['dropoff_latitude'].astype(str)
taxi['url']="http://router.project-osrm.org/route/v1/driving/"+taxi['pickup_location']+";"+taxi['dropoff_location']+"?geometries=geojson&overview=simplified&steps=false"
taxi['index1'] = taxi.index


In [8]:
#taxi1=taxi.tail(800000)
url1=taxi[['index1','id','url']].values.tolist()
urls=[ tuple(i) for i in url1]
print(len(urls))

1


In [9]:
def get_direction(idx,idee,url):
        
                json_list=[]
                
                headers = {'Pengchong Tang': 'my-app for kaggle.com competition'}
                response = requests.get(url,headers=headers)
                direction=response.content.decode('utf-8')

                try:
                    var=json.loads(direction)
                    distance=var['routes'][0]['distance']
                    duration=var['routes'][0]['duration']
                    json_list.append({'id': idee,'distance': distance,'duration':duration })
                except KeyError:
                    json_list.append({'id': idee })
        
        
                
                if idx % 1000==0:
                      print(str(idx)+" done")

                return json_list

In [17]:
response = requests.get(url)
direction=response.content.decode('utf-8')

In [18]:
direction

'{"code":"NoRoute"}'

In [10]:
import multiprocessing
 
pool = multiprocessing.Pool(100)  
   
result=pool.starmap(get_direction,urls)
pool.close()


0 done


In [11]:
result1=[ x[0] for x in result]
direction_train=pd.DataFrame(result1)
direction_train.to_csv("direction.csv")